# שלב 2 – עיבוד מקדים לטופיק מודלינג בעברית (UGC בלבד)

מחברת זו מניחה שכבר הרצת את **שלב 1 – הכנת נתונים** ושיש לך את הקבצים תחת `data/`:

- `data/hebert_ugc_docs.parquet`
- `data/hebert_ugc_sample_10k.parquet`

כאן נעשה:

1. הגדרת נתיבים (`data/`, `models/`).
2. טעינת הדאטה של UGC.
3. ניקוי טקסט בסיסי לעברית (URLs, אמוג׳ים, סימני פיסוק וכו').
4. טוקניזציה פשוטה + הסרת מילות עצירה.
5. שמירת גרסאות "נקיות" לשימוש במחברות המודלים (שלב 3–4).


# ועכשיו מסודר!


מטרת השלב: להפוך טקסטים גולמיים לייצוג נקי ואחיד שמוכן למודלים.

**זרימת העבודה בקצרה:**
1. ניקוי טקסט בסיסי → `clean_text`
2. למטיזציה (DictaBERT‑Lex) → `lemma_text_raw`
3. הסרת stopwords → `lemma_text`
4. שמירת `processed_*.parquet`


### 1. התקנה וייבוא ספריות


In [ ]:
!pip install -U transformers==4.39.3 accelerate torch tqdm pandas requests "huggingface_hub>=0.23.0"


### 2. הגדרות נתיבים וחיבור לדרייב


In [ ]:
import os
import re
import requests
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from tqdm.auto import tqdm
from transformers import AutoModelForTokenClassification, AutoTokenizer

In [ ]:
import sys
from pathlib import Path

def _add_repo_root():
    cwd = Path.cwd()
    for path in [cwd] + list(cwd.parents):
        if (path / 'project_paths.py').exists():
            sys.path.insert(0, str(path))
            return path
    return cwd

_add_repo_root()

try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception:
    pass

from project_paths import get_paths

paths = get_paths().ensure()
BASE_DIR = paths.root
DATA_DIR = paths.data_dir
MODELS_DIR = paths.models_dir
RESULTS_DIR = paths.results_dir
RESULTS_ROOT = RESULTS_DIR

print('BASE_DIR  :', BASE_DIR)
print('DATA_DIR  :', DATA_DIR)
print('MODELS_DIR:', MODELS_DIR)
print('RESULTS_DIR:', RESULTS_DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 3. בדיקת חומרה


> טיפ: למטיזציה עם DictaBERT‑Lex כבדה יחסית. על CPU מומלץ להקטין `batch_size` או לעבוד על דגימה.


In [ ]:
# בדיקת GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚙️ עובד על: {device}")

⚙️ עובד על: cuda


### 4. טעינת מודל הלימטיזציה DictaBERT-Lex


In [ ]:
DICTA_LEMMA_MODEL_NAME = "dicta-il/dictabert-lex"

tokenizer = AutoTokenizer.from_pretrained(DICTA_LEMMA_MODEL_NAME)
model = torch.hub.load(
    "dicta-il/dictabert-lex", "model", trust_repo=True
) if False else None
# ↑ לא נשתמש בזה. נשתמש ב-trust_remote_code דרך transformers:

from transformers import AutoModel
model = AutoModel.from_pretrained(DICTA_LEMMA_MODEL_NAME, trust_remote_code=True)

model.to(device)
model.eval()
print("✅ DictaBERT-Lex loaded")


### 5. פונקציות עזר לעיבוד טקסט (ניקוי, מילות עצירה, למטיזציה)


In [ ]:
def get_stopwords():
    url = "https://raw.githubusercontent.com/NNLP-IL/Stop-Words-Hebrew/main/stopswords_list_short.txt"
    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        base = r.text.splitlines()
    except Exception:
        base = ['את', 'של', 'על', 'עם', 'כי', 'היה', 'כל', 'או', 'אלא', 'אם', 'לא']

    ugc = [
        'ממש','פשוט','כאילו','בכלל','לגמרי','הייתי','היה','תהיה','הזה','הזאת',
        'אני','אתה','אנחנו','הם','היא','הוא','רק','גם','עוד','אז','שלא',
        'מה','מי','יש','אין','טוב','סבבה','אחלה','וואלה','בקיצור','כמובן',
        'דרך','אגב','נראה','רואה','צריך','יכול','רוצה','חושב','אומר',
        'לא','כן','אל','זה','זאת','אלה','כזה','הן',
        'או','עם','של','על','את',
        'px','שמאל','ימין','ממוזער','thumbnail','jpg','png','gif'
    ]

    cleaned_base = []
    for w in base:
        w = w.strip()
        if not w or w.startswith("#"):
            continue
        w = re.sub(r"[^\u0590-\u05FF]", "", w)  # רק עברית
        if w:
            cleaned_base.append(w.lower())

    stop = set(cleaned_base) | set(x.strip().lower() for x in ugc if x.strip())
    # תיקוני למטיזציה נפוצים
    stop |= {"היי", "כול"}
    return stop

STOPWORDS = get_stopwords()
print("✅ STOPWORDS:", len(STOPWORDS), "| sanity:", "של" in STOPWORDS, "על" in STOPWORDS, "את" in STOPWORDS, "או" in STOPWORDS)

def remove_stopwords(text):
    if not isinstance(text, str):
        return ""
    out = []
    for w in text.split():
        lw = w.strip().lower()
        if not lw:
            continue
        if lw in STOPWORDS:
            continue
        if len(lw) <= 1:
            continue
        out.append(lw)
    return " ".join(out)


✅ STOPWORDS: 187 | sanity: True True True True


In [ ]:
def clean_text_basic(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"[^\u0590-\u05FF\s\.,?!]", " ", text)  # עברית + פיסוק
    text = re.sub(r"\s+", " ", text)
    return text.strip()


In [ ]:
def lemmatize_batch(texts, batch_size=8, show_progress=True, empty_cache_every=25):
    results = []
    rng = range(0, len(texts), batch_size)
    if show_progress:
        rng = tqdm(rng, desc="🧠 Lemmatization (batches)", leave=True)

    use_amp = (device.type == "cuda")

    for step, i in enumerate(rng, start=1):
        batch = texts[i:i + batch_size]

        with torch.inference_mode():
            try:
                if use_amp:
                    with torch.autocast(device_type="cuda", dtype=torch.float16):
                        preds = model.predict(batch, tokenizer)
                else:
                    preds = model.predict(batch, tokenizer)
            except TypeError:
                preds = []
                for t in batch:
                    if use_amp:
                        with torch.autocast(device_type="cuda", dtype=torch.float16):
                            preds.append(model.predict([t], tokenizer)[0])
                    else:
                        preds.append(model.predict([t], tokenizer)[0])

        for pairs in preds:
            lemmas = []
            for _, lemma in pairs:
                if not isinstance(lemma, str):
                    continue
                low = lemma.strip().lower()
                if not low or low in ("[blank]", "blank", "[unk]", "unk"):
                    continue
                lemmas.append(low)
            results.append(" ".join(lemmas))

        if device.type == "cuda" and (step % empty_cache_every == 0):
            torch.cuda.empty_cache()
            gc.collect()

    return results


### 6. הרצת תהליך העיבוד על מערכי הנתונים


In [ ]:
def cleanup_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def process_dataset(input_path, output_name, batch_size=8):
    print(f"\n🚀 מתחיל עיבוד עבור: {output_name}...")

    if not input_path.exists():
        print(f"❌ קובץ לא נמצא: {input_path}")
        return

    with tqdm(total=1, desc="📥 Load parquet") as pbar:
        df = pd.read_parquet(input_path)
        pbar.update(1)
    print(f"   נטענו {len(df)} מסמכים.")

    print("   🛁 ניקוי בסיסי...")
    df["clean_text"] = df["text"].progress_apply(clean_text_basic)
    df = df[df["clean_text"].str.len() > 5].reset_index(drop=True)

    print("   🧠 למטיזציה...")
    df["lemma_text_raw"] = lemmatize_batch(
        df["clean_text"].tolist(),
        batch_size=batch_size,
        show_progress=True
    )

    print("   🧹 הסרת stopwords...")
    df["lemma_text"] = df["lemma_text_raw"].progress_apply(remove_stopwords)

    # סינון סופי + שמירה
    df = df.drop(columns=["lemma_text_raw"])
    df = df[df["lemma_text"].str.len() > 0].reset_index(drop=True)

    save_path = DATA_DIR / output_name
    df.to_parquet(save_path)

    print(f"✅ נשמר: {save_path} | rows={len(df)}")
    print("   דוגמה lemma:", df["lemma_text"].iloc[0][:120], "...")


In [ ]:
cleanup_memory()

WIKI_RAW_PATH = DATA_DIR / "wiki_25k_truncated.parquet"
process_dataset(WIKI_RAW_PATH, "processed_wiki.parquet", batch_size=8)

cleanup_memory()

UGC_RAW_PATH = DATA_DIR / "hebert_ugc_ready.parquet"
process_dataset(UGC_RAW_PATH, "processed_ugc.parquet", batch_size=8)

cleanup_memory()
print("\n🎉 כל הדאטה עובד ונשמר בשני קבצים נפרדים!")



🚀 מתחיל עיבוד עבור: processed_wiki.parquet...


📥 Load parquet:   0%|          | 0/1 [00:00<?, ?it/s]

   נטענו 25000 מסמכים.
   🛁 ניקוי בסיסי...


  0%|          | 0/25000 [00:00<?, ?it/s]

   🧠 למטיזציה...


...

   🧠 למטיזציה...


🧠 Lemmatization (batches):   0%|          | 0/639 [00:00<?, ?it/s]

   🧹 הסרת stopwords...


  0%|          | 0/5112 [00:00<?, ?it/s]

✅ נשמר: /content/drive/MyDrive/HebrewTopicModel/data/processed_ugc.parquet | rows=5098
   דוגמה lemma: תור בל מסכה למה בא לי נשאר בית תשובה אישה נדחף ביקש התרחק ...

🎉 כל הדאטה עובד ונשמר בשני קבצים נפרדים!


### 7. אימות ובדיקת דוגמאות אחרי עיבוד


In [ ]:
import pandas as pd

# הגדרות תצוגה כדי שנוכל לקרוא את הטקסט המלא
pd.set_option('display.max_colwidth', 150)
pd.set_option('display.max_rows', 10)

def show_processing_examples(file_name, n=3):
    file_path = DATA_DIR / file_name

    if not file_path.exists():
        print(f"⚠️ הקובץ {file_name} לא נמצא. האם הרצת את העיבוד?")
        return

    print(f"\n🔍 --- בדיקת דוגמאות מתוך: {file_name} ---")
    df_temp = pd.read_parquet(file_path)

    # בחירת דגימה אקראית
    sample = df_temp.sample(n)

    for idx, row in sample.iterrows():
        print(f"\n🔹 מסמך #{idx}:")
        print(f"   1. ORIGINAL:  {row['text'][:150]}...")
        print(f"   2. CLEAN:     {row['clean_text'][:150]}...")
        print(f"   3. LEMMA:     {row['lemma_text'][:150]}...")
        print("-" * 80)

# --- הרצה ---
# הצגת דוגמאות מויקיפדיה (אמור להיות רשמי ונקי)
show_processing_examples("processed_wiki.parquet")

# הצגת דוגמאות מ-UGC (אמור להיות סלנג, והלמות צריכות לתקן את זה)
show_processing_examples("processed_ugc.parquet")


🔍 --- בדיקת דוגמאות מתוך: processed_wiki.parquet ---

🔹 מסמך #7978:
   1. ORIGINAL:  בלבואה הוא שמו של המטבע הרשמי של פנמה. שמו ניתן לו לכבוד וסקו נונייס דה בלבואה, מגלה הארצות (קונקיסטאדור) הספרדי. לפי תקן ISO 4217 סימנו הוא PAB. הבלב...
   2. CLEAN:     בלבואה הוא שמו של המטבע הרשמי של פנמה. שמו ניתן לו לכבוד וסקו נונייס דה בלבואה, מגלה הארצות קונקיסטאדור הספרדי. לפי תקן סימנו הוא . הבלבואה מוצמד לדול...
   3. LEMMA:     שם מטבע רשמי פנמה שם ניתן לכבוד דה מגלה ארץ ספרדי תקן סימן מוצמד דולר אמריקאי שער חליפין נתן המרה יחס פנמה התחלק מטבע משקל ממד הרכב מתכת מקביל ארץ ברי...
--------------------------------------------------------------------------------

🔹 מסמך #9693:
   1. ORIGINAL:  שמאל| דיסק באופנוע שמאל| דיסק מאוורר במכונית שמאל| תוף מפורק. בצד שמאל ניתן לראות התוף ומצד ימין את הרפידות בֶּלֶם (בעגה: בְּרֵקְס; מאנגלית: brakes) ה...
   2. CLEAN:     שמאל דיסק באופנוע שמאל דיסק מאוורר במכונית שמאל תוף מפורק. בצד שמאל ניתן לראות התוף ומצד ימין את הרפידות בֶּלֶם בעגה בְּרֵקְס מאנגלית 

### 8. תיקון מילות עצירה ובדיקת השפעה


In [ ]:
fix_existing_processed("processed_wiki.parquet", "processed_wiki_fixed.parquet")
fix_existing_processed("processed_ugc.parquet", "processed_ugc_fixed.parquet")



🔧 Fixing stopwords for: processed_wiki.parquet (rows=24999)


  0%|          | 0/24999 [00:00<?, ?it/s]

✅ saved: /content/drive/MyDrive/HebrewTopicModel/data/processed_wiki_fixed.parquet

🔧 Fixing stopwords for: processed_ugc.parquet (rows=5099)


  0%|          | 0/5099 [00:00<?, ?it/s]

✅ saved: /content/drive/MyDrive/HebrewTopicModel/data/processed_ugc_fixed.parquet


In [ ]:
from collections import Counter

def print_top_words(series, top_n=40, title=""):
    c = Counter()
    for t in series.dropna().astype(str):
        for w in t.split():
            if w:
                c[w] += 1

    print(f"\n🔎 {title} | Top-{top_n}")
    for i, (w, cnt) in enumerate(c.most_common(top_n), start=1):
        print(f"{i:>2}. {w}: {cnt}")


In [ ]:
def print_topwords_from_processed(file_name, top_n=40):
    path = DATA_DIR / file_name
    df = pd.read_parquet(path)
    print(f"\n📄 {file_name} | rows={len(df)}")

    if "text" in df.columns:
        print_top_words(df["text"], top_n=top_n, title="ORIGINAL")
    if "clean_text" in df.columns:
        print_top_words(df["clean_text"], top_n=top_n, title="CLEAN")
    if "lemma_text" in df.columns:
        print_top_words(df["lemma_text"], top_n=top_n, title="LEMMA (post-stopwords)")

# דוגמה
print_topwords_from_processed("processed_wiki.parquet", top_n=40)
print_topwords_from_processed("processed_ugc.parquet", top_n=40)



📄 processed_wiki.parquet | rows=24999

🔎 ORIGINAL | Top-40
 1. של: 62964
 2. על: 28444
 3. הוא: 26136
 4. את: 24967
 5. היא: 14758
 6. או: 11463
 7. היה: 10886
 8. –: 9863
 9. בשנת: 9337
10. עם: 9202
11. ידי: 8011
12. גם: 7952
13. בין: 7874
14. שמאל|: 7647
15. -: 5222
16. לאחר: 5000
17. עד: 4530
18. נולד: 4197
19. הייתה: 4149
20. זה: 4021
21. ביותר: 4013
22. אשר: 3982
23. הם: 3916
24. שם: 3837
25. לא: 3820
26. אך: 3802
27. (באנגלית:: 3657
28. כל: 3645
29. אחד: 3103
30. העיר: 3034
31. ישראל: 2969
32. היסטוריה: 2934
33. הראשון: 2828
34. מספר: 2805
35. זו: 2736
36. המאה: 2731
37. בית: 2584
38. בו: 2447
39. יותר: 2443
40. ביוגרפיה: 2440

🔎 CLEAN | Top-40
 1. של: 63036
 2. על: 28664
 3. הוא: 26212
 4. את: 25001
 5. ,: 16624
 6. היא: 14807
 7. או: 12237
 8. .: 11650
 9. היה: 10902
10. בשנת: 9375
11. עם: 9282
12. גם: 8142
13. שמאל: 8042
14. ידי: 8027
15. ב: 8001
16. בין: 7961
17. ה: 5199
18. לאחר: 5042
19. שם: 4923
20. נולד: 4764
21. עד: 4607
22. באנגלית: 4280
23. הייתה: 4156
24. ביותר: 4125

### 9. ניתוח מילים שכיחות


In [ ]:
from collections import Counter
import pandas as pd

def top_counter(series):
    c = Counter()
    for t in series.dropna().astype(str):
        for w in t.split():
            if w:
                c[w] += 1
    return c

def print_counter(c, top_n=40, title=""):
    print(f"\n🔎 {title} | Top-{top_n}")
    for i, (w, cnt) in enumerate(c.most_common(top_n), start=1):
        print(f"{i:>2}. {w}: {cnt}")

def compare_stopwords_on_processed(file_name, top_n=40):
    path = DATA_DIR / file_name
    if not path.exists():
        print("❌ לא נמצא:", path)
        return

    df = pd.read_parquet(path)
    print(f"\n📄 {file_name} | rows={len(df)}")

    # BEFORE: מה שיש עכשיו בקובץ
    before_series = df["lemma_text"].fillna("")
    before_c = top_counter(before_series)

    # AFTER: מחשבים מחדש lemma_text עם ה-STOPWORDS המתוקנים
    after_series = before_series.progress_apply(remove_stopwords)
    after_c = top_counter(after_series)

    print_counter(before_c, top_n=top_n, title="BEFORE (current lemma_text)")
    print_counter(after_c, top_n=top_n, title="AFTER (re-applied remove_stopwords)")

    # מה “נפל” הכי הרבה
    drops = []
    for w, cnt_before in before_c.items():
        cnt_after = after_c.get(w, 0)
        if cnt_before > cnt_after:
            drops.append((w, cnt_before - cnt_after, cnt_before, cnt_after))
    drops.sort(key=lambda x: x[1], reverse=True)

    print(f"\n⬇️ הכי ירדו (Top-{top_n})  |  format: word: dropped (before → after)")
    for i, (w, d, b, a) in enumerate(drops[:top_n], start=1):
        print(f"{i:>2}. {w}: {d} ({b} → {a})")

    # אופציונלי: אם תרצה גם לשמור את העמודה המתוקנת (לא שומר אוטומטית)
    return after_series

# --- הרצה ---
compare_stopwords_on_processed("processed_wiki.parquet", top_n=40)
compare_stopwords_on_processed("processed_ugc.parquet", top_n=40)



📄 processed_wiki.parquet | rows=24999


  0%|          | 0/24999 [00:00<?, ?it/s]


🔎 BEFORE (current lemma_text) | Top-40
 1. של: 69259
 2. על: 32624
 3. את: 31081
 4. שנה: 26758
 5. היי: 17561
 6. שם: 15176
 7. או: 12361
 8. עם: 10897
 9. יד: 10298
10. בין: 9999
11. ראשון: 9375
12. יום: 8525
13. יותר: 8410
14. אחד: 7950
15. רב: 7802
16. ישראל: 7361
17. בית: 7321
18. ארץ: 7210
19. בן: 6918
20. חי: 6441
21. עיר: 6433
22. ספר: 6347
23. מדינה: 6317
24. נולד: 6136
25. לאחר: 5939
26. עד: 5836
27. עולם: 5549
28. גדול: 5344
29. חלק: 5249
30. כל: 5218
31. מאה: 5083
32. מספר: 5025
33. משפחה: 4918
34. מלחמה: 4787
35. שניים: 4655
36. קבוצה: 4642
37. אנגלית: 4617
38. אשר: 4495
39. שונה: 4385
40. ברית: 4282

🔎 AFTER (re-applied remove_stopwords) | Top-40
 1. שנה: 26758
 2. היי: 17561
 3. שם: 15176
 4. יד: 10298
 5. ראשון: 9375
 6. יום: 8525
 7. אחד: 7950
 8. רב: 7802
 9. ישראל: 7361
10. בית: 7321
11. ארץ: 7210
12. בן: 6918
13. חי: 6441
14. עיר: 6433
15. ספר: 6347
16. מדינה: 6317
17. נולד: 6136
18. עולם: 5549
19. גדול: 5344
20. חלק: 5249
21. מאה: 5083
22. משפחה: 4918
23. מלחמה: 4

  0%|          | 0/5099 [00:00<?, ?it/s]


🔎 BEFORE (current lemma_text) | Top-40
 1. את: 4090
 2. של: 2373
 3. על: 2030
 4. כול: 1608
 5. עם: 1278
 6. כל: 1206
 7. היי: 1159
 8. מדינה: 848
 9. איש: 778
10. אם: 747
11. קורונה: 744
12. בית: 739
13. ממשלה: 724
14. ביבי: 683
15. חולה: 647
16. יום: 643
17. בריאות: 609
18. יותר: 598
19. אחד: 549
20. אבל: 515
21. סגר: 499
22. לי: 497
23. ישראל: 485
24. לו: 485
25. כמו: 459
26. שנה: 456
27. או: 448
28. שר: 418
29. חי: 404
30. למה: 386
31. שם: 372
32. כבר: 364
33. כסף: 362
34. ראש: 358
35. משרד: 358
36. קיבל: 335
37. אחר: 326
38. זמן: 325
39. הרבה: 323
40. עובד: 320

🔎 AFTER (re-applied remove_stopwords) | Top-40
 1. כול: 1608
 2. היי: 1159
 3. מדינה: 848
 4. איש: 778
 5. קורונה: 744
 6. בית: 739
 7. ממשלה: 724
 8. ביבי: 683
 9. חולה: 647
10. יום: 643
11. בריאות: 609
12. אחד: 549
13. סגר: 499
14. לי: 497
15. ישראל: 485
16. שנה: 456
17. שר: 418
18. חי: 404
19. למה: 386
20. שם: 372
21. כבר: 364
22. כסף: 362
23. ראש: 358
24. משרד: 358
25. קיבל: 335
26. זמן: 325
27. עובד: 320
28. עולם: 31

,lemma_text
0,תור בל מסכה למה בא לי נשאר תשובה אישה נדחף ביקש התרחק
1,אשם ביבי חרד מיליארד כיסא אפשר פושע חשוד מולדת המשיך נמלט משפט כחלון סך אדם חסר השכלה מתאים תפקיד שר אוצר שר התאים ביבי נבחר
2,"קורה ביבי עסוק ספין עצם ניהול הוכחה ביב דיבור גרוע ניהול ארגון ככה מל""ל עסוק יחס ציבור ניהל באמת"
3,מלכתחילה שולם אזרח ישראל חוב ילד שילם
4,נתניהו אוהב ארץ כול ארץ צומת ישראל אמיתי עובד שירת צבא משלם מס פרזיטים מוצץ דם חסר דעת עריק נהג מונית שוק רמלה אוכל חינם בן ברק התגייסות הדהים גיר...
...,...
5094,שילם קריסה מערכה בריאות ראשות מלך ביבי שר בריאות שנה היי שר בריאות מערכת קורס מדינה סגר מערכה בריאות מורעל משומן ערוך כלום גל שני נתן היי נמנע תוכ...
5095,התבייש תאב בצע כסף גב עכשיו בוכה עובד בן ערובה בהמה תאב בצע
5096,התחיל התרגל שר אוצר ראש ממשלה מחליט פקידות התחיל איבד כוח הראה קורה פקיד מרגיש קובע מדיניות בפועל פקיד אוצר פקיד משרד משפט כבוד עושה עבודה מאומץ ה...
5097,נתן כסף מעון ראש ממשלה בנייה מטוס פרטי הוד מעלה מלך ביבי המשיך תחזק בית פרטי קיסריה המשיך סיפק בן רכב נהג מאבטח בן גדול הפקיר עובד משק כסף מס ממנה...


## סכימת קבצי הפלט
נוצרים שני קבצים עיקריים:
- `processed_wiki.parquet`
- `processed_ugc.parquet`

עמודות חשובות:
- `text` – הטקסט המקורי
- `clean_text` – ניקוי בסיסי
- `lemma_text` – למטיזציה + הסרת stopwords

המחברות בשלב 3–5 מניחות שהקבצים האלה קיימים תחת `DATA_DIR`.


In [ ]:
from collections import Counter

def compute_df(series):
    """
    מחזיר Counter: מילה -> מספר מסמכים שבהם הופיעה
    """
    df_counter = Counter()
    total_docs = len(series)

    for text in series.dropna().astype(str):
        words = set(text.split())  # חשוב: set → מסמך אחד נספר פעם אחת
        for w in words:
            df_counter[w] += 1

    return df_counter, total_docs


In [ ]:
def print_high_df(df_counter, total_docs, top_n=30, min_ratio=0.4, title=""):
    print(f"\n🔥 מילים עם DF גבוה | {title}")
    print(f"(מופיעות בלפחות {int(min_ratio*100)}% מהמסמכים)\n")

    items = []
    for w, df in df_counter.items():
        ratio = df / total_docs
        if ratio >= min_ratio:
            items.append((w, df, ratio))

    items.sort(key=lambda x: x[2], reverse=True)

    for i, (w, df, ratio) in enumerate(items[:top_n], start=1):
        print(f"{i:>2}. {w}: {df} מסמכים ({ratio:.1%})")


In [ ]:
def print_low_df(df_counter, top_n=30, max_df=2, title=""):
    print(f"\n🧊 מילים עם DF נמוך | {title}")
    print(f"(מופיעות ב־{max_df} מסמכים או פחות)\n")

    items = [(w, df) for w, df in df_counter.items() if df <= max_df]
    items.sort(key=lambda x: x[1])

    for i, (w, df) in enumerate(items[:top_n], start=1):
        print(f"{i:>2}. {w}: {df}")


In [ ]:
df_wiki = pd.read_parquet(DATA_DIR / "processed_wiki_fixed.parquet")

df_counter_wiki, N_wiki = compute_df(df_wiki["lemma_text"])

print_high_df(
    df_counter_wiki,
    N_wiki,
    min_ratio=0.4,
    title="WIKI"
)

print_low_df(
    df_counter_wiki,
    max_df=2,
    title="WIKI"
)



🔥 מילים עם DF גבוה | WIKI
(מופיעות בלפחות 40% מהמסמכים)

 1. שנה: 14013 מסמכים (56.1%)
 2. היי: 10144 מסמכים (40.6%)

🧊 מילים עם DF נמוך | WIKI
(מופיעות ב־2 מסמכים או פחות)

 1. הוויקי: 1
 2. מיקור: 1
 3. בלאו: 1
 4. אינשטיין: 1
 5. טאב: 1
 6. הלייזר: 1
 7. ראוטר: 1
 8. שכבתי: 1
 9. אלכסיס: 1
10. תדפיס: 1
11. בולד: 1
12. פאזי: 1
13. אובלי: 1
14. התחרות: 1
15. חתרנית: 1
16. בייסיק: 1
17. דיגיטציה: 1
18. עראק: 1
19. הורוסקופ: 1
20. שקריות: 1
21. מאירסון: 1
22. מתריס: 1
23. נאנח: 1
24. תשאלי: 1
25. תווי: 1
26. חנך: 1
27. אינפי: 1
28. האמפרי: 1
29. משהה: 1
30. ##ט: 1


In [ ]:
df_ugc = pd.read_parquet(DATA_DIR / "processed_ugc_fixed.parquet")

df_counter_ugc, N_ugc = compute_df(df_ugc["lemma_text"])

print_high_df(
    df_counter_ugc,
    N_ugc,
    min_ratio=0.4,
    title="UGC"
)

print_low_df(
    df_counter_ugc,
    max_df=2,
    title="UGC"
)



🔥 מילים עם DF גבוה | UGC
(מופיעות בלפחות 40% מהמסמכים)


🧊 מילים עם DF נמוך | UGC
(מופיעות ב־2 מסמכים או פחות)

 1. מל"ל: 1
 2. דחף: 1
 3. מתחדש: 1
 4. אפור: 1
 5. מושפל: 1
 6. לוואי: 1
 7. שיטתי: 1
 8. המצאה: 1
 9. העסקה: 1
10. רפיון: 1
11. ##נלי: 1
12. בג: 1
13. איפא: 1
14. משכנע: 1
15. תחרותי: 1
16. אינדיבידואלי: 1
17. חודר: 1
18. מכחיש: 1
19. נקווה: 1
20. מתארח: 1
21. דאטה: 1
22. אביגדור: 1
23. פארמה: 1
24. קייט: 1
25. מחייב: 1
26. הרואי: 1
27. שעל: 1
28. התקצר: 1
29. חקוק: 1
30. פקפק: 1
